# House Prices Competition : Term Project 

### Importing Libraries:

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn import neighbors
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

import matplotlib.pyplot as plt
plt.style.use(style='ggplot')
plt.rcParams['figure.figsize'] = (10, 6)

In [13]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
import lightgbm as lgb


In [14]:
# getting data from feature engineering notebook
%store -r train_set_1
%store -r test_set_1
%store -r ytrain_1

train_set = train_set_1
test_set = test_set_1

In [15]:
train_set.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,...,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Street_Grvl,Street_Pave
0,856,854,0,0,3,706.0,0.0,1.0,0,150.0,...,0,0,0,0,0,0,0,1,0,1
1,1262,0,0,0,3,978.0,0.0,0.0,1,284.0,...,0,0,0,0,0,0,0,1,0,1
2,920,866,0,0,3,486.0,0.0,1.0,0,434.0,...,0,0,0,0,0,0,0,1,0,1
3,961,756,0,0,3,216.0,0.0,1.0,0,540.0,...,0,0,0,0,0,0,0,1,0,1
4,1145,1053,0,0,4,655.0,0.0,1.0,0,490.0,...,0,0,0,0,0,0,0,1,0,1


In [16]:
print ("Train data shape:", train_set.shape)

Train data shape: (1452, 291)


In [17]:
test_set.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,...,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Street_Grvl,Street_Pave
1452,896,0,0,0,2,468.0,144.0,0.0,0,270.0,...,0,0,0,0,0,0,0,1,0,1
1453,1329,0,0,0,3,923.0,0.0,0.0,0,406.0,...,0,0,0,0,0,0,0,1,0,1
1454,928,701,0,0,3,791.0,0.0,0.0,0,137.0,...,0,0,0,0,0,0,0,1,0,1
1455,926,678,0,0,3,602.0,0.0,0.0,0,324.0,...,0,0,0,0,0,0,0,1,0,1
1456,1280,0,0,0,2,263.0,0.0,0.0,0,1017.0,...,0,0,0,0,0,0,0,1,0,1


In [18]:
print ("Test data shape:", test_set.shape)


Test data shape: (1459, 291)


## Building the model:

In [19]:
train_set = train_set.select_dtypes(include=[np.number]).interpolate().dropna()
test_set = test_set.select_dtypes(include=[np.number]).interpolate().dropna()
train_set.head(5)

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,...,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Street_Grvl,Street_Pave
0,856,854,0,0,3,706.0,0.0,1.0,0,150.0,...,0,0,0,0,0,0,0,1,0,1
1,1262,0,0,0,3,978.0,0.0,0.0,1,284.0,...,0,0,0,0,0,0,0,1,0,1
2,920,866,0,0,3,486.0,0.0,1.0,0,434.0,...,0,0,0,0,0,0,0,1,0,1
3,961,756,0,0,3,216.0,0.0,1.0,0,540.0,...,0,0,0,0,0,0,0,1,0,1
4,1145,1053,0,0,4,655.0,0.0,1.0,0,490.0,...,0,0,0,0,0,0,0,1,0,1


In [20]:
from sklearn import preprocessing
#num_columns_train = train_set.select_dtypes(include = [np.number]).columns.values.tolist()
#num_columns_test = test_set.select_dtypes(include = [np.number]).columns.values.tolist()

#scaler = StandardScaler()

#train_set.loc[:,num_columns_train] = scaler.fit_transform(train_set[num_columns_train])
#test_set.loc[:,num_columns_test] = scaler.fit_transform(test_set[num_columns_test])


#print(train_set.head())
#print(test_set.head())

In [21]:
#y = train_set.SalePrice


y = np.log(ytrain_1)
X = train_set#.drop(['SalePrice'] , axis = 1)

### PCA:

In [22]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)                
t_train = scaler.transform(X)
pca_hp = PCA(30)
x_fit = pca_hp.fit_transform(t_train)
np.exp(pca_hp.explained_variance_ratio_)

array([ 1.0736579 ,  1.03167676,  1.02673246,  1.02388901,  1.02276886,
        1.01835836,  1.01627782,  1.01571604,  1.012281  ,  1.01216943,
        1.01185297,  1.01123428,  1.01064432,  1.01007107,  1.00988119,
        1.00967377,  1.00924598,  1.00921448,  1.00889524,  1.00865794,
        1.0083885 ,  1.00817255,  1.00802755,  1.00789791,  1.00772193,
        1.00760914,  1.00756966,  1.00746309,  1.0072716 ,  1.0071893 ])

train_test_split() returns four objects:

* X_train is the subset of our features used for training.
* X_test is the subset which will be our 'hold-out' set - what we'll use to test the model.
* y_train is the target variable SalePrice which corresponds to X_train.
* y_test is the target variable SalePrice which corresponds to X_test. 

random_state=42 allow reproducible results.

In [23]:
from sklearn.preprocessing import MinMaxScaler


X_train, X_test, y_train, y_test = train_test_split(
                                   X, y, random_state=42, test_size=.33)

from sklearn.preprocessing import StandardScaler
#scalerX = MinMaxScaler().fit(X_train)
#scalery = MinMaxScaler().fit(y_train.reshape(-1,1))
#X_train = scalerX.transform(X_train)
#y_train = scalery.transform(y_train.reshape(-1,1))
#X_test = scalerX.transform(X_test)
#y_test = scalery.transform(y_test.reshape(-1,1))


### Random Forest:

In [24]:
from sklearn.ensemble import RandomForestRegressor

# Fit Random Forest on Training Set
regressor = RandomForestRegressor(n_estimators=300, random_state=0)
model_random_forest = regressor.fit(X_train, y_train)


# Score model
print ("Test score is: ", model_random_forest.score(X_test, y_test))

Test score is:  0.875852404421


In [25]:
predictions_1 = model_random_forest.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions_1)))

Mean squared Error : 0.019184894702


### KNN:

In [26]:
# find best k number

ourScore=[]
for nn in range(1,15):
    knn = neighbors.KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                                        metric_params=None, n_jobs=1, n_neighbors=nn,p=2,
                                        weights='uniform')
    knn.fit(X_train, y_train)
    train_score = knn.score(X_train, y_train)
    test_score = knn.score(X_test, y_test)
    #print('k: %d, Train Acc: %.3f, Test Acc: %.3f' % (nn, train_score, test_score))
    rowScore=[nn,train_score,test_score]
    ourScore.append(rowScore)

In [27]:
k=5
knn = neighbors.KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=k, p=2,
          weights='uniform')
knn.fit(X_train, y_train)
train_score = knn.score(X_train, y_train)
test_score = knn.score(X_test, y_test)

print('k: %d, Train Acc: %.3f, Test Acc: %.3f' % (k, train_score, test_score))

k: 5, Train Acc: 0.804, Test Acc: 0.680


In [28]:
predictions_2 = knn.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions_2)))

Mean squared Error : 0.0494293297457


### Linear Regression:

In [29]:
# Linear Regression
from sklearn.linear_model import LinearRegression 

regressor = LinearRegression() 
regressor.fit(X_train, y_train) 
print ("Test score is: \n", regressor.score(X_test, y_test))

Test score is: 
 -345840.431275


In [30]:
predictions_3 = regressor.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions_3)))

Mean squared Error : 53443.8980607


### Linear Regression with ridge regularization:

In [31]:
#to check what alpha value is better for the model

for i in range (-5, 5):
    alpha = 10**i
    rm = linear_model.Ridge(alpha=alpha)
    ridge_model = rm.fit(X_train, y_train)
    preds_ridge = ridge_model.predict(X_test)

    plt.scatter(preds_ridge, y_test, alpha=.75, color='b')
    plt.xlabel('Predicted Price')
    plt.ylabel('Actual Price')
    plt.title('Ridge Regularization with alpha = {}'.format(alpha))
    overlay = 'R^2 is: {}\nRMSE is: {}'.format(
                    ridge_model.score(X_test, y_test),
                    mean_squared_error(y_test, preds_ridge))
    plt.annotate(s=overlay,xy=(12.1,10.6),size='x-large')
    #plt.show()

In [32]:
from sklearn.metrics import mean_squared_error

linm = linear_model.Ridge(alpha = 10)

linm.fit(X_train, y_train)
print ("Test score is: \n", linm.score(X_test, y_test))

Test score is: 
 0.901239708666


In [33]:
predictions = linm.predict(X_test)
print("Mean Squared Error : " + str(mean_squared_error(y_test,predictions)))

Mean Squared Error : 0.0152617195777


### MLP with PCA:

In [34]:
#mlp_regressor = MLPRegressor(solver='lbfgs',hidden_layer_sizes = (200,5,5),alpha = 1.0, activation = 'relu', max_iter = 100)
#mlp_regressor.fit(X_train,y_train)

#print ("Test score is: \n", mlp_regressor.score(X_test, y_test))

In [35]:
#predictions7 = mlp_regressor.predict(X_test1)
#print("Mean Squared Error : " + str(mean_squared_error(y_test,predictions7)))

### Basic Decision tree:

In [36]:
# Define model
basic_decision_tree = DecisionTreeRegressor()

# Fit model
basic_decision_tree.fit(X, y)

print("R^2 is: \n", basic_decision_tree.score(X_test, y_test))

R^2 is: 
 0.999999996944


In [37]:
predictions_5 = basic_decision_tree.predict(X_test)
print("Mean Squared Error : " + str(mean_squared_error(y_test,predictions_5)))

Mean Squared Error : 4.72276373045e-10


### XGBoost

In [38]:
learning_rates = np.arange(0.0, 0.09, 0.0015)
learning_rates
best_learning_rates = [0.0885, 0.0735, 0.0705, 0.0615, 0.06, 0.0585, 0.057, 0.0555]

In [39]:
# to find best parameters
from xgboost import XGBRegressor

actual_values = y_test

for i in best_learning_rates:
    n_estimators = 1000
    my_model = XGBRegressor(n_estimators=n_estimators,learning_rate=i)
    xgboost_model = my_model.fit(X_train, y_train, early_stopping_rounds=5, 
                 eval_set=[(X_test, y_test)], verbose=False)
    preds_xgboost = xgboost_model.predict(X_test)

    plt.scatter(preds_xgboost, actual_values, alpha=.75, color='b')
    plt.xlabel('Predicted Price')
    plt.ylabel('Actual Price')
    plt.title('XGBoost with  = {}'.format(n_estimators))
    overlay = 'R^2 is: {}\nRMSE is: {}\nlearning Rate is: {}'.format(
                        xgboost_model.score(X_test, y_test),
                        mean_squared_error(y_test, preds_xgboost),
                        i)
    plt.annotate(s=overlay,xy=(12.1,10.6),size='x-large')
    #plt.show()

In [40]:
# XGBoost
XGBoost = XGBRegressor(n_estimators = 1000,learning_rate=0.0585)
XGBoost.fit(X_train, y_train)


print ("Test Score: \n", XGBoost.score(X_test, y_test))

Test Score: 
 0.905846405165


In [41]:
prediction = XGBoost.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,prediction)))

Mean squared Error : 0.0145498331587


### Lasso regression:

In [42]:
# Lasso regression
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
lasso.fit(X_train, y_train)
print ("Test Score: \n", lasso.score(X_test, y_test))

Test Score: 
 0.904965675904


In [43]:
predictions_lasso = lasso.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions_lasso)))

Mean squared Error : 0.0146859348533


### Elastic Net Regression:

In [44]:
# Elastic Net regression
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
ENet.fit(X_train, y_train)
print ("Test Score: \n", ENet.score(X_test, y_test))

Test Score: 
 0.90495230902


In [45]:
predictions_ENet = ENet.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions_ENet)))

Mean squared Error : 0.0146880004772


### Gradient Boosting Regression:

In [46]:
# Gradient boosting regression
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

GBoost.fit(X_train, y_train)
print ("Test Score: \n", GBoost.score(X_test, y_test))

Test Score: 
 0.900061935524


In [47]:
predictions_GBoost = GBoost.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions_GBoost)))

Mean squared Error : 0.0154437243408


### Light GBM:

In [48]:
# LightGBM
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
model_lgb.fit(X_train, y_train)
print ("Test Score: \n", model_lgb.score(X_test, y_test))

Test Score: 
 0.902791193686


In [49]:
predictions_model_lgb = model_lgb.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions_model_lgb)))

Mean squared Error : 0.0150219640142


### Avereging Models:

In [50]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone


class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

### Enet regression + GBoost + LGBM + Lasso regression (Best Score):

In [51]:
#Best version
averaged_models = AveragingModels(models = (ENet, GBoost, model_lgb, lasso))

averaged_models.fit(X_train, y_train)
print ("Test Score: \n", averaged_models.score(X_test, y_test))

Test Score: 
 0.910253536225


In [52]:
predictions = averaged_models.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions)))

Mean squared Error : 0.0138687861764


### Bagging method:

In [60]:
from sklearn.ensemble import BaggingRegressor

bagging = BaggingRegressor(linear_model.Ridge(alpha = 10),max_samples=0.5, max_features=0.5)
bagging.fit(X_train, y_train)
print ("Test Score: \n", bagging.score(X_test, y_test))


Test Score: 
 0.891953670326


In [59]:
predictions_b = bagging.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions_b)))

Mean squared Error : 0.0158728093359


In [61]:
from sklearn.ensemble import BaggingRegressor

bagging_1 = BaggingRegressor(GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5),max_samples=0.5, max_features=0.5)
bagging_1.fit(X_train, y_train)
print ("Test Score: \n", bagging.score(X_test, y_test))


Test Score: 
 0.891953670326


In [62]:
predictions_b1 = bagging.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions_b1)))

Mean squared Error : 0.0166967185153


In [64]:
from sklearn.ensemble import BaggingRegressor

bagging_2 = BaggingRegressor(lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11),max_samples=0.5, max_features=0.5)
bagging_2.fit(X_train, y_train)
print ("Test Score: \n", bagging.score(X_test, y_test))


Test Score: 
 0.891953670326


In [65]:
predictions_b2 = bagging.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions_b2)))

Mean squared Error : 0.0166967185153


### Avreging bagging models:

In [67]:
#Best version
averaged_models_2 = AveragingModels(models = (bagging, bagging_1, bagging_2, linm))

averaged_models_2.fit(X_train, y_train)
print ("Test Score: \n", averaged_models.score(X_test, y_test))

Test Score: 
 0.910253536225


In [68]:
predictions = averaged_models_2.predict(X_test)
print("Mean squared Error : " + str(mean_squared_error(y_test,predictions)))

Mean squared Error : 0.0151193134258


### Stacking :
Using Meta model with cross validation

### Making a submission

In [ ]:
submission = pd.DataFrame()
%store -r test_ID
submission['Id'] = test_ID

In [ ]:
feats = test_set.select_dtypes(
        include=[np.number])
feats.shape

In [ ]:
predictions = stacked_averaged_models.predict(feats)
predictions
#final_predictions = np.exp(predictions)

#final_predictions = scalery.inverse_transform(predictions.reshape(-1,1))


In [ ]:
#submission['SalePrice'] = final_predictions
#submission.head(5)

In [ ]:
#submission.to_csv("normalize_submission.csv", index=False)